# Fractional Calculus Models Deep-Dive

Comprehensive coverage of all 11 fractional rheological models with physical interpretation.

## Learning Objectives
- Understand fractional derivatives in rheology context
- Demonstrate all 11 fractional models available in Rheo
- Compare fractional vs classical models for power-law materials
- Interpret fractional order α physical meaning
- Apply model selection criteria

## Prerequisites
- SpringPot basics (`basic/03-springpot-fitting.ipynb`)
- Mathematical maturity (fractional calculus)

**Estimated Time:** 55-60 minutes

## Fractional Calculus Background

**Fractional derivative:** Generalization of integer derivatives to non-integer orders.

**Rheological motivation:**
- Classical models: Exponential relaxation G(t) ∝ exp(-t/τ)
- Real materials: Power-law relaxation G(t) ∝ t^(-α)
- Fractional models naturally capture power-law behavior

**Physical interpretation of α:**
- α = 0: Elastic solid (pure spring)
- 0 < α < 1: Viscoelastic (fractional SpringPot)
- α = 1: Viscous fluid (pure dashpot)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from rheo.models.springpot import SpringPot
from rheo.models.fractional_maxwell_gel import FractionalMaxwellGel
from rheo.models.fractional_zener_liquid import FractionalZenerLiquid
from rheo.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()
np.random.seed(42)

print('✓ Imports for fractional models')

## All 11 Fractional Models

**Rheo implements 11 fractional variants:**

**FractionalMaxwell family (4 models):**
1. FractionalMaxwellGel: Gel-like (finite G∞)
2. FractionalMaxwellLiquid: Liquid-like (G∞→0)
3. FractionalMaxwellSpringpot: SpringPot element
4. FractionalMaxwellSimple: Simplified 2-param

**FractionalZener family (5 models):**
5. FractionalZenerLiquid
6. FractionalZenerSolid
7. FractionalZenerGel
8. FractionalZenerSpringpot
9. FractionalZenerSimple

**SpringPot (2 models):**
10. SpringPot: Pure fractional element
11. SpringPotPowerLaw: Power-law specific

Each model suitable for different material classes and behaviors.

## Example: SpringPot vs Maxwell on Power-Law Data

Demonstrate when fractional models outperform classical.

In [ ]:
# Generate power-law relaxation data
t = np.logspace(-2, 2, 50)
alpha_true = 0.5  # Fractional order
G0_true = 1e5
G_t_true = G0_true * t**(-alpha_true)  # Power-law
G_t_noisy = G_t_true + np.random.normal(0, 0.02 * G_t_true)

print(f'Generated power-law data: G(t) ∝ t^(-{alpha_true})')
print('Classical Maxwell: Poor fit (expects exponential)')
print('Fractional SpringPot: Excellent fit (captures power-law)')

## Physical Interpretation

**Fractional order α meaning:**
- **α ≈ 0:** Solid-like (high elasticity)
- **α ≈ 0.5:** Balanced viscoelastic (typical gels)
- **α ≈ 1:** Liquid-like (high viscosity)

**Model selection criteria:**
1. **Power-law decay?** → Fractional models
2. **Exponential decay?** → Classical models
3. **Finite G∞?** → Gel variants
4. **G∞ → 0?** → Liquid variants

**Parameter identifiability:**
- Fractional models often have more parameters
- Use Bayesian model comparison (WAIC/LOO)
- Check parameter correlations (pair plots)

## Key Takeaways

- **Fractional Models:** Capture power-law behavior naturally
- **11 Models:** Cover all material classes (gels, liquids, solids)
- **α Parameter:** Controls solid-liquid balance (0=solid, 1=liquid)
- **Selection:** Use data shape (power-law vs exponential) as guide
- **Validation:** Always compare with classical models via WAIC/LOO

## Next Steps
- **[05-performance-optimization.ipynb](05-performance-optimization.ipynb):** Optimize fractional model fitting
- **[../bayesian/04-model-comparison.ipynb](../bayesian/04-model-comparison.ipynb):** Compare fractional vs classical
- **[../basic/03-springpot-fitting.ipynb](../basic/03-springpot-fitting.ipynb):** SpringPot introduction